<a href="https://colab.research.google.com/github/karimekirey/redesneuronales/blob/main/Confiabilidad_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto Final de Redes Neuronales Artificiales
"Confiabilidad de Productos en Amazon Basado en reseñas"

Equipo 7:
Karime Kirey Sánchez Andrade 1914174
Lázaro Gael Cortez Moncada



---



PREPROCESAMIENTO DE LOS DATOS
---



In [54]:
import pandas as pd
import numpy as np
from collections import Counter

# el dataset puede obtenerse del siguiente link https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt
# Importamos el dataset utilizado contiene reseñas de artículos de la categoría de Oficina. Especificamos que los encabezados están en la fila 0

dtf = pd.read_table('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', header=0, on_bad_lines='skip')
df=pd.DataFrame(dtf) #Convertimos el dataset en un Dataframe con pandas
print(df.head())

df.shape
df.info()

<ipython-input-54-5378f73491aa>:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dtf = pd.read_table('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', header=0, on_bad_lines='skip')


  marketplace  customer_id       review_id  product_id  product_parent  \
0          US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1          US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2          US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3          US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4          US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   

                                       product_title product_category  \
0     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet  Office Products   
1          Dust-Off Compressed Gas Duster, Pack of 4  Office Products   
2  Amram Tagger Standard Tag Attaching Tagging Gu...  Office Products   
3  AmazonBasics 12-Sheet High-Security Micro-Cut ...  Office Products   
4  Derwent Colored Pencils, Inktense Ink Pencils,...  Office Products   

  star_rating  helpful_votes  total_votes vine verified_purchase  \
0           5            0.0          0.0    N  

In [55]:
#print(df.columns)

In [56]:
#En esta parte del código se borraron las columnas que no se utilizarán con la función de pandas "drop"
#labeling the columns
columnas_quitar = ['marketplace', 'customer_id', 'review_id', 'product_parent', 'vine']
datos=df.drop(columnas_quitar, axis=1)
print(datos.head(20))


    product_id                                      product_title  \
0   B001BM2MAC     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet   
1   B00DZYEXPQ          Dust-Off Compressed Gas Duster, Pack of 4   
2   B00RTMUHDW  Amram Tagger Standard Tag Attaching Tagging Gu...   
3   B00D7H8XB6  AmazonBasics 12-Sheet High-Security Micro-Cut ...   
4   B001XCWP34  Derwent Colored Pencils, Inktense Ink Pencils,...   
5   B004J2NBCO  Quartet Magnetic Dry-Erase Weekly Organizer, 6...   
6   B00MOPAG8K  KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...   
7   B003AHIK7U              Bible Dry Highlighting Kit (Set of 4)   
8   B006TKH2RO  Parker Ingenuity Large Black Rubber & Metal CT...   
9   B00W61M9K0                                RFID Card Protector   
10  B00BXOGI3A  The Original Stand Steady Standing Desk - Inst...   
11  B0018RHWGE  Zebra Sparky-1 WKP1-10C Fluorescent Marker (10...   
12  B00IA8H2OG  E-Z Ink (TM) Compatible Ink Cartridge Replacem...   
13  B007TOIA66                   E

In [59]:
print("Datos del preprocesamiento")
datos = datos.iloc[1:, :]
from IPython.display import display
display(datos.head(10))


Datos del preprocesamiento


,product_id,product_title,product_category,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date
1,B00DZYEXPQ,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,B00RTMUHDW,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,B00D7H8XB6,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,B001XCWP34,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31
5,B004J2NBCO,"Quartet Magnetic Dry-Erase Weekly Organizer, 6...",Office Products,5,0.0,0.0,Y,Five Stars,Perfect for planning weekly meals. Removrd the...,2015-08-31
6,B00MOPAG8K,KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...,Office Products,5,0.0,0.0,N,Five Stars,Gold plated fusers are the best! It will never...,2015-08-31
7,B003AHIK7U,Bible Dry Highlighting Kit (Set of 4),Office Products,5,6.0,6.0,Y,Won't bleed through pages!,I have used these highlighters for my bible fo...,2015-08-31
8,B006TKH2RO,Parker Ingenuity Large Black Rubber & Metal CT...,Office Products,5,0.0,0.0,Y,Nice pen!,Heavy pen that writes very well. I've had it ...,2015-08-31
9,B00W61M9K0,RFID Card Protector,Office Products,5,0.0,0.0,Y,Credit card sleeves,Not sure if they work but sent quickly and fit...,2015-08-31
10,B00BXOGI3A,The Original Stand Steady Standing Desk - Inst...,Office Products,4,0.0,0.0,N,Four Stars,Great product for the price. Easy to put toget...,2015-08-31


In [80]:
#limpieza de palabras como "is", "the"

import re
import string
from gensim.parsing.preprocessing import remove_stopwords

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = remove_stopwords(text)
    return text

In [82]:
datos['review_body'] = datos['review_body'].fillna('').astype(str) #algunas reseñas contienen valores numéricos. Se convierte todo a str para 
datos['review_body'] = datos['review_body'].apply(clean_text)

display(datos.head(10)) #imprime el review_body ya limpio, sin puntuación y sin palabras de relleno

,product_id,product_title,product_category,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date
1,B00DZYEXPQ,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",whats commodity item come worldbr having need ...,2015-08-31
2,B00RTMUHDW,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,Y,but I am sure I will like it.,havent sure like,2015-08-31
3,B00D7H8XB6,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,Y,and the shredder was dirty and the bin was par...,labeled received clearly box previously opened...,2015-08-31
4,B001XCWP34,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,Y,Four Stars,gorgeous colors easy use,2015-08-31
5,B004J2NBCO,"Quartet Magnetic Dry-Erase Weekly Organizer, 6...",Office Products,5,0.0,0.0,Y,Five Stars,perfect planning weekly meals removrd want con...,2015-08-31
6,B00MOPAG8K,KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...,Office Products,5,0.0,0.0,N,Five Stars,gold plated fusers best fail,2015-08-31
7,B003AHIK7U,Bible Dry Highlighting Kit (Set of 4),Office Products,5,6.0,6.0,Y,Won't bleed through pages!,highlighters bible year love highlight alot bi...,2015-08-31
8,B006TKH2RO,Parker Ingenuity Large Black Rubber & Metal CT...,Office Products,5,0.0,0.0,Y,Nice pen!,heavy pen writes ive couple weeks received com...,2015-08-31
9,B00W61M9K0,RFID Card Protector,Office Products,5,0.0,0.0,Y,Credit card sleeves,sure work sent quickly fits wallet slots,2015-08-31
10,B00BXOGI3A,The Original Stand Steady Standing Desk - Inst...,Office Products,4,0.0,0.0,N,Four Stars,great product price easy stable easily adjustable,2015-08-31


In [ ]:
#NORMALIZACIÓN DE DATOS NUMÉRICOS



---
DIVISIÓN SET DE ENTRENAMIENTO Y VALIDACIÓN
---

